In [1]:
# Explore

In [12]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import sys

from lime import lime_image
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

import input_data
import models

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:

def settings(sample_rate=16000, clip_duration_ms=1000.0,
                    window_size_ms=30.0, window_stride_ms=10.0,
                    feature_bin_count=40, preprocess='average', n_classes=12):

    m = models.prepare_model_settings(n_classes, sample_rate,
                                      clip_duration_ms, window_size_ms, window_stride_ms,
                                      feature_bin_count, preprocess)
    return m

model_settings = settings()

def wav_to_features(input_wav):
    
    audio_processor = input_data.AudioProcessor(None, None, 0, 0, '', 0, 0, model_settings, None)

    results = audio_processor.get_features_for_wav(input_wav, model_settings, session)
    features = results[0]
    return features

f = wav_to_features('../../data/speech_dataset/nine/122c5aa7_nohash_0.wav')
f.shape

ff 43 98


(1, 98, 43, 1)

In [44]:


image_size = (98, 43, 1)
f_size = (None, 98, 43, 1)

tf.reset_default_graph()

processed_images = tf.placeholder(tf.float32, shape=f_size)
model = models.create_model(processed_images, model_settings,
                            model_architecture='low_latency_conv', is_training=False)

def transform_img_fn(path_list):
    out = []
    for f in path_list:
        image = wav_to_features(f).squeeze(axis=0)
        print('s', image.shape, type(image))
        i = tf.convert_to_tensor(image) 
        out.append(i)
    return session.run([out])[0]

session = tf.Session()

#tf.get_variable_scope().reuse_variables()

checkpoint = 'train/low_latency_conv.ckpt-12600'

models.load_variables_from_checkpoint(session, checkpoint)
saver = tf.train.Saver(tf.global_variables())
saver.restore(session, checkpoint)

probabilities = model
def predict_fn(paths):
    images = transform_img_fn(paths)
    return session.run(probabilities, feed_dict={processed_images: images})

predict_fn(['../../data/speech_dataset/nine/122c5aa7_nohash_0.wav'])


conv out 36 1 186
---------
Variables: name (type shape) [size]
---------
first_weights:0 (float32_ref 98x8x1x186) [145824, bytes: 583296]
first_bias:0 (float32_ref 186) [186, bytes: 744]
first_fc_weights:0 (float32_ref 6696x128) [857088, bytes: 3428352]
first_fc_bias:0 (float32_ref 128) [128, bytes: 512]
second_fc_weights:0 (float32_ref 128x128) [16384, bytes: 65536]
second_fc_bias:0 (float32_ref 128) [128, bytes: 512]
final_fc_weights:0 (float32_ref 128x12) [1536, bytes: 6144]
final_fc_bias:0 (float32_ref 12) [12, bytes: 48]
Total size of variables: 1021286
Total bytes of variables: 4085144
INFO:tensorflow:Restoring parameters from train/low_latency_conv.ckpt-12600
INFO:tensorflow:Restoring parameters from train/low_latency_conv.ckpt-12600
s (98, 43, 1) <class 'numpy.ndarray'>


array([[-0.68610454,  0.09988894, -0.04474275,  0.2969219 , -0.18442798,
         0.4439205 , -0.05753042,  0.03617053,  0.20074761, -0.1346468 ,
         0.01625966,  0.00969209]], dtype=float32)